In [1]:
import json
import curlify
import requests

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import pandas as pd

pd.set_option('display.max_colwidth', None)

bearer_token = "AAAAAAAAAAAAAAAAAAAAADSXjAEAAAAAQ32sCpTWcilVX%2BQj0BjOArOSYCE%3DUax6ZCizLRwwQcWQirExGQEDEB903dxBZKM4LAuGTMTvMa4jVP"

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/crarojasca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
s = requests.Session()
def sendQuery(query, max_results=None, next_token=None):
    url = "https://api.twitter.com/2/tweets/search/recent"
    
    headers = {
        "Authorization": "Bearer {}".format(bearer_token)
    }
    
    params = {
        'query': query,
    #     'start_time': start_date,
    #     'end_time': end_date,
        'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
        'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
        'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
        'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
        'next_token': {}
    }
    
    if max_results: params['max_results'] = max_results
    if next_token: params['next_token'] = next_token

    req = requests.Request(
        "GET",
        url,
        params=params,
        headers=headers
    ).prepare()
    
#     print(curlify.to_curl(req))
    response = s.send(req)
    results = json.loads(response.text)
    return results
    
start_time = "2021-03-01T00:00:00.000Z"
end_time = "2021-03-31T00:00:00.000Z"


def extract_data(query):
    data = pd.DataFrame()

    next_token = None
    while True:
        results = sendQuery(query=query, next_token=next_token)
        data = pd.concat([data, pd.DataFrame(results["data"])], ignore_index=True)
        if not "next_token" in results["meta"]:
            break
        next_token = results["meta"]["next_token"]    
    
    columns = list(data.columns)
    columns.remove("text")
    columns.remove("public_metrics")
    data = data[["text", "public_metrics"] + columns]
    
    return data


In [3]:
data = extract_data("from:ClimateScam")
data

,text,public_metrics,edit_history_tweet_ids,conversation_id,id,author_id,reply_settings,created_at,lang,source,referenced_tweets,in_reply_to_user_id
0,"More MIT &amp; earth's thermometer ( below): they've tracked global temperature fluctuations through geologic history. ""We have data going back 66 million years, &amp; applied the mathematical theory of stochastic differential equations."" OK? You want to argue with them?!! #climatescam","{'retweet_count': 8, 'reply_count': 3, 'like_count': 25, 'quote_count': 0}",[1594370139784613891],1594370139784613891,1594370139784613891,1453033532000022530,everyone,2022-11-20T16:40:37.000Z,en,Twitter Web App,NaN,NaN
1,"Negotiators at COP 27 agreed to fund poor countries (including China) coping with ""climate disasters."" Nobody agreed to the $. MIT announced their scientists discovered earth's internal ""thermometer"" prevents an overheating or overcooling planet! We've saved $!!? #climatescam","{'retweet_count': 7, 'reply_count': 3, 'like_count': 30, 'quote_count': 0}",[1594366610873147394],1594366610873147394,1594366610873147394,1453033532000022530,everyone,2022-11-20T16:26:36.000Z,en,Twitter Web App,NaN,NaN
2,"Climate anxiety, brought on by the likes of Al Gore &amp; Dr Moore, is crippling the youth of the world. #ClimateScam https://t.co/mPiUkbVVhO","{'retweet_count': 45, 'reply_count': 7, 'like_count': 136, 'quote_count': 1}",[1594333303095558149],1594333303095558149,1594333303095558149,1453033532000022530,everyone,2022-11-20T14:14:15.000Z,en,Twitter for Android,"[{'type': 'quoted', 'id': '1594322869059829761'}]",NaN
3,Hang onto your hats kids....#climatescam https://t.co/xG9ZKPSB8t,"{'retweet_count': 211, 'reply_count': 26, 'like_count': 405, 'quote_count': 14}",[1593955935965298690],1593955935965298690,1593955935965298690,1453033532000022530,everyone,2022-11-19T13:14:44.000Z,en,Twitter for Android,NaN,NaN
4,RT @JunkScience: Today's estimated global warming is 0.36°C (0.65°F) warmer than the 1979-2000 average for the date... despite 50% of total…,"{'retweet_count': 2359, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",[1593927560836038668],1593927560836038668,1593927560836038668,1453033532000022530,everyone,2022-11-19T11:21:58.000Z,en,Twitter for Android,"[{'type': 'retweeted', 'id': '1593620122803048448'}]",NaN
5,"@BTCgroover Excellent, @BTCgroover!! So many people have been sucked into the Climate Change BS. The truly shocking thing is that the Scam has been going as long as it has. Looking forward to keeping the anti-scam message going! #climatescam","{'retweet_count': 2, 'reply_count': 0, 'like_count': 2, 'quote_count': 0}",[1593716174960852993],1593708780805660675,1593716174960852993,1453033532000022530,everyone,2022-11-18T21:22:00.000Z,en,Twitter Web App,"[{'type': 'replied_to', 'id': '1593711582374027264'}]",865855010499952640
6,"The energy crisis = unrealistic policies based on unscientific conclusions re: climate. Coal, oil and nat. gas made nations wealthy enough to ignore Net Zero hype, which produces high energy prices &amp; shortages. Now, gov't subsidies go to wind &amp; solar rat holes. SCAM #climatescam","{'retweet_count': 11, 'reply_count': 2, 'like_count': 27, 'quote_count': 0}",[1593714186814689281],1593714186814689281,1593714186814689281,1453033532000022530,everyone,2022-11-18T21:14:06.000Z,en,Twitter Web App,NaN,NaN
7,"World leaders and officials attending the United Nations COP27 climate conference will spend up to $100/entree to eat red meat, seafood &amp; other gourmet menu items. Note: the UN has discouraged red meat consumption because beef farming = carbon emissions. Really. #climatescam","{'retweet_count': 35, 'reply_count': 7, 'like_count': 91, 'quote_count': 2}",[1593708780805660675],1593708780805660675,1593708780805660675,1453033532000022530,everyone,2022-11-18T20:52:37.000Z,en,Twitter Web App,NaN,NaN
8,"Germany’s energy supply is crashing, &amp; its wind industry is “sliding into a crisis.” The German gov't aims to solve the country’s energy woe

In [1]:
results["includes"]

NameError: name 'results' is not defined

In [ ]:
data = extract_data("#ClimateScam #COP27")
data = data.drop_duplicates("text")
data

In [ ]:
all_stopwords = stopwords.words('english')
def count_words(col):    
    df = col.str.split(expand=True).stack().value_counts().reset_index()
    df.columns = ['Word', 'Frequency'] 
    df = df[~df["Word"].isin(all_stopwords)]
    return df

count_words(data.text)[:30]

In [ ]:
results = sendQuery(
    query="retweets_of_tweet_id:1590785323105423360"
)
results

In [ ]:
results = sendQuery(
    query="from:ABCFactCheck"
)
results

In [ ]:
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

In [5]:
import tweepy

auth = tweepy.OAuth1UserHandler(
   consumer_key, consumer_secret, access_token, access_token_secret
)

api = tweepy.API(auth)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

NameError: name 'consumer_key' is not defined